## debug the transformation problem

```bash
File "/home/r7user3/ZhangHT/github/maskrcnn-benchmark/maskrcnn_benchmark/engine/trainer.py", line 118, in do_train
    for iteration, (images, targets, _) in enumerate(data_loader, start_iter):
  File "/home/r7user3/anaconda2/envs/maskrcnn/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 615, in __next__
    batch = self.collate_fn([self.dataset[i] for i in indices])
  File "/home/r7user3/anaconda2/envs/maskrcnn/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 615, in <listcomp>
    batch = self.collate_fn([self.dataset[i] for i in indices])
  File "/home/r7user3/ZhangHT/github/maskrcnn-benchmark/maskrcnn_benchmark/data/datasets/coco.py", line 61, in __getitem__
    img, target = self.transforms(img, target)
  File "/home/r7user3/ZhangHT/github/maskrcnn-benchmark/maskrcnn_benchmark/data/transforms/transforms.py", line 19, in __call__
    image, target = t(image, target)
  File "/home/r7user3/ZhangHT/github/maskrcnn-benchmark/maskrcnn_benchmark/data/transforms/transforms.py", line 95, in __call__
    target=F.rotate(target,angle)
  File "/home/r7user3/anaconda2/envs/maskrcnn/lib/python3.7/site-packages/torchvision/transforms/functional.py", line 593, in rotate
    raise TypeError('img should be PIL Image. Got {}'.format(type(img)))
TypeError: img should be PIL Image. Got <class 'maskrcnn_benchmark.structures.bounding_box.BoxList'>

```

### train_net.py

In [1]:
from maskrcnn_benchmark.config import cfg
import os
import torch
from maskrcnn_benchmark.data import make_data_loader

In [2]:
# simulate the true working dir
os.chdir('/home/r7user3/ZhangHT/github/maskrcnn-benchmark/')

In [3]:
cfg.merge_from_file('./configs/e2e_mask_rcnn_R_50_FPN_1x.yaml')
cfg.freeze()

In [4]:
distributed=False
start_iter=0

In [7]:
data_loader = make_data_loader(
        cfg,
        is_train=True,
        is_distributed=distributed,
        start_iter=start_iter,
    )

When using more than one image per GPU you may encounter an out-of-memory (OOM) error if your GPU does not have sufficient memory. If this happens, you can reduce SOLVER.IMS_PER_BATCH (for training) or TEST.IMS_PER_BATCH (for inference). For training, you must also adjust the learning rate and schedule length according to the linear scaling rule. See for example: https://github.com/facebookresearch/Detectron/blob/master/configs/getting_started/tutorial_1gpu_e2e_faster_rcnn_R-50-FPN.yaml#L14


loading annotations into memory...
Done (t=0.37s)
creating index...
index created!


### trainer.py

In [6]:
dataiter=enumerate(data_loader, start_iter)
_,(images,targets,_)=next(dataiter)

In [7]:
target=targets[0]

In [11]:
#mask=target.get_field('masks')

In [22]:
#mask.polygons[0].polygons

[tensor([125., 116., 124., 117., 123., 117., 122., 118., 122., 119., 122., 120.,
         122., 121., 122., 122., 122., 123., 122., 124., 123., 124., 124., 125.,
         125., 125., 126., 125., 127., 125., 128., 125., 129., 124., 129., 123.,
         129., 122., 128., 121., 127., 120., 126., 119., 126., 118., 126., 117.])]

In [14]:
a=torch.Tensor(1,28,28)
b=a.shape

In [18]:
tuple(b)

(1, 28, 28)

## multi train

```bash
CUDA_VISIBLE_DEVICES=2,3 python -m torch.distributed.launch --nproc_per_node=2 --master_addr 127.0.0.2 --master_port 29501 tools/train_net.py --config-file configs/e2e_mask_rcnn_R_50_FPN_1x.yaml SOLVER.IMS_PER_BATCH 4 SOLVER.BASE_LR 0.005 SOLVER.MAX_ITER 30000 SOLVER.STEPS "(15000, 20000)" TEST.IMS_PER_BATCH 2
```

```bash
python tools/train_net.py --config-file "configs/e2e_mask_rcnn_R_50_FPN_1x.yaml" SOLVER.IMS_PER_BATCH 2 SOLVER.BASE_LR 0.0025 SOLVER.MAX_ITER 60000 SOLVER.STEPS "(30000, 40000)" TEST.IMS_PER_BATCH 1
```

In [9]:
from pycocotools.coco import COCO
from maskrcnn_benchmark.config import cfg
import os
import torch
from maskrcnn_benchmark.data import make_data_loader

In [5]:
coco=COCO('./train.json')

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


In [ ]:
# simulate the true working dir
os.chdir('/home/r7user3/ZhangHT/github/maskrcnn-benchmark/')

In [26]:
a=torch.rand(2,3,256,256)
a.dim()

4

In [27]:
a.shape

torch.Size([2, 3, 256, 256])

In [29]:
print(a.sum())

tensor(196345.0781)


In [3]:
True or False

True